In [1]:
from google.colab import drive
# Mount your google drive to the /content/drive directory.
# This ensures that files changes remain saved in your drive.
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/cpen455-project/

/content/drive/MyDrive/cpen455-project


In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
import time
import os
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
import wandb
from tqdm import tqdm
from pprint import pprint
import argparse
from pytorch_fid.fid_score import calculate_fid_given_paths
import logging
import pandas as pd
import numpy as np

import dotenv
dotenv.load_dotenv()

True

In [5]:
import importlib
import utils, model, dataset, generation_evaluation, classification_evaluation

importlib.reload(utils)  # Reload the script after changes
importlib.reload(model)  # Reload the script after changes
importlib.reload(dataset)  # Reload the script after changes
importlib.reload(generation_evaluation)  # Reload the script after changes
importlib.reload(classification_evaluation)  # Reload the script after changes

from utils import *
from model import *
from dataset import *
import generation_evaluation
import classification_evaluation

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.WARN)

In [ ]:
from dataclasses import dataclass

@dataclass
class Config:
    en_wandb: bool = False
    tag: str = "default"
    sampling_interval: int = 5
    data_dir: str = "data"
    save_dir: str = "models"
    sample_dir: str = "samples"
    dataset: str = "cpen455"
    save_interval: int = 10
    load_params: str = None
    obs: tuple = (3, 32, 32)
    nr_resnet: int = 1
    nr_filters: int = 40
    nr_logistic_mix: int = 5
    lr: float = 0.0002
    lr_decay: float = 0.999995
    batch_size: int = 64
    sample_batch_size: int = 32
    base_epoch: int = 0
    max_epochs: int = 5000
    seed: int = 1

In [ ]:
args = Config(
    batch_size=16,
    sample_batch_size=16,
    sampling_interval=5,
    save_interval=10,
    max_epochs=50,
    en_wandb=True,
    nr_filters=40,
    nr_logistic_mix=5,
    nr_resnet=5,
    tag="Middle_f40_l5_r5",
    sample_dir="samples_f40_l5_r5",
    save_dir="models_f40_l5_r5",
)

pprint(args.__dict__)
check_dir_and_create(args.save_dir)

# reproducibility
torch.manual_seed(args.seed)
np.random.seed(args.seed)

model_name = 'pcnn_' + args.dataset + "_"
model_path = args.save_dir + '/'
if args.load_params is not None:
    model_name = model_name + 'load_model'
    model_path = model_path + model_name + '/'
else:
    model_name = model_name + 'from_scratch'
    model_path = model_path + model_name + '/'

job_name = "PCNN_Training_" + "dataset:" + args.dataset + "_" + args.tag

if args.en_wandb:
    # start a new wandb run to track this script
    wandb.init(
        # set entity to specify your username or team name
        # entity="qihangz-work",
        # set the wandb project where this run will be logged
        project="CPEN455HW",
        # group=Group Name
        name=job_name,
    )
    wandb.config.current_time = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))
    wandb.config.update(args)

#set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#Reminder: if you have patience to read code line by line, you should notice this comment. here is the reason why we set num_workers to 0:
#In order to avoid pickling errors with the dataset on different machines, we set num_workers to 0.
#If you are using ubuntu/linux/colab, and find that loading data is too slow, you can set num_workers to 1 or even bigger.
kwargs = {'num_workers':2, 'pin_memory':True, 'drop_last':True}

{'batch_size': 16,
 'data_dir': 'data',
 'dataset': 'cpen455',
 'en_wandb': True,
 'load_params': None,
 'lr': 0.0002,
 'lr_decay': 0.999995,
 'max_epochs': 50,
 'nr_filters': 40,
 'nr_logistic_mix': 5,
 'nr_resnet': 5,
 'obs': (3, 32, 32),
 'sample_batch_size': 16,
 'sample_dir': 'samples_f40_l5_r5',
 'sampling_interval': 5,
 'save_dir': 'models_f40_l5_r5',
 'save_interval': 10,
 'seed': 1,
 'tag': 'Middle_f40_l5_r5'}


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: parshan-pjavanrood (parshan-pjavanrood-university-of-british-columbia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# set data
if "mnist" in args.dataset:
    ds_transforms = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor(), rescaling, replicate_color_channel])
    train_loader = torch.utils.data.DataLoader(datasets.MNIST(args.data_dir, download=True,
                        train=True, transform=ds_transforms), batch_size=args.batch_size,
                            shuffle=True, **kwargs)

    test_loader  = torch.utils.data.DataLoader(datasets.MNIST(args.data_dir, train=False,
                    transform=ds_transforms), batch_size=args.batch_size, shuffle=True, **kwargs)

elif "cifar" in args.dataset:
    ds_transforms = transforms.Compose([transforms.ToTensor(), rescaling])
    if args.dataset == "cifar10":
        train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(args.data_dir, train=True,
            download=True, transform=ds_transforms), batch_size=args.batch_size, shuffle=True, **kwargs)

        test_loader  = torch.utils.data.DataLoader(datasets.CIFAR10(args.data_dir, train=False,
                    transform=ds_transforms), batch_size=args.batch_size, shuffle=True, **kwargs)
    elif args.dataset == "cifar100":
        train_loader = torch.utils.data.DataLoader(datasets.CIFAR100(args.data_dir, train=True,
            download=True, transform=ds_transforms), batch_size=args.batch_size, shuffle=True, **kwargs)

        test_loader  = torch.utils.data.DataLoader(datasets.CIFAR100(args.data_dir, train=False,
                    transform=ds_transforms), batch_size=args.batch_size, shuffle=True, **kwargs)
    else:
        raise Exception('{} dataset not in {cifar10, cifar100}'.format(args.dataset))

elif "cpen455" in args.dataset:
    ds_transforms = transforms.Compose([transforms.Resize((32, 32)), rescaling])
    train_loader = torch.utils.data.DataLoader(CPEN455Dataset(root_dir=args.data_dir,
                                                              mode = 'train',
                                                              transform=ds_transforms),
                                                batch_size=args.batch_size,
                                                shuffle=True,
                                                **kwargs)
    test_loader  = torch.utils.data.DataLoader(CPEN455Dataset(root_dir=args.data_dir,
                                                              mode = 'test',
                                                              transform=ds_transforms),
                                                batch_size=args.batch_size,
                                                shuffle=True,
                                                **kwargs)
    val_loader  = torch.utils.data.DataLoader(CPEN455Dataset(root_dir=args.data_dir,
                                                              mode = 'validation',
                                                              transform=ds_transforms),
                                                batch_size=args.batch_size,
                                                shuffle=True,
                                                **kwargs)
else:
    raise Exception('{} dataset not in {mnist, cifar, cpen455}'.format(args.dataset))

In [ ]:
def train_or_test(model, data_loader, optimizer, loss_op, device, args, epoch, mode = 'training'):
    logging.debug('mode: {}'.format(mode))
    if mode == 'training':
        model.train()
    else:
        model.eval()

    deno =  args.batch_size * np.prod(args.obs) * np.log(2.)
    loss_tracker = mean_tracker()

    for batch_idx, item in enumerate(tqdm(data_loader)):
        logging.debug('batch_idx: {}'.format(batch_idx))
        model_input, label = item
        model_input = model_input.to(device)
        model_output = model(model_input, label)

        loss = loss_op(model_input, model_output)
        loss_tracker.update(loss.item()/deno)
        if mode == 'training':
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        elif mode == 'val':
            pred_labels = classification_evaluation.get_label(model, model_input, device)
            org_labels = torch.tensor([my_bidict[l] for l in label], device=device)
            acc = torch.sum(org_labels == pred_labels) / len(label)
            if args.en_wandb:
                wandb.log({mode + "-Classification-Acc" : acc})

    if args.en_wandb:
        wandb.log({mode + "-Average-BPD" : loss_tracker.get_mean()})
        wandb.log({mode + "-epoch": epoch})


In [ ]:
args.obs = (3, 32, 32)
input_channels = args.obs[0]

loss_op   = lambda real, fake : discretized_mix_logistic_loss(real, fake)
sample_op = lambda x : sample_from_discretized_mix_logistic(x, args.nr_logistic_mix)

In [ ]:
## Start from scratch

model = PixelCNN(nr_resnet=args.nr_resnet, nr_filters=args.nr_filters,
            input_channels=input_channels, nr_logistic_mix=args.nr_logistic_mix)
model = model.to(device)

if args.load_params:
    model.load_state_dict(torch.load(args.load_params))
    print('model parameters loaded')

optimizer = optim.Adam(model.parameters(), lr=args.lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=args.lr_decay)

for epoch in tqdm(range(args.max_epochs)):
    train_or_test(model = model,
                  data_loader = train_loader,
                  optimizer = optimizer,
                  loss_op = loss_op,
                  device = device,
                  args = args,
                  epoch = epoch,
                  mode = 'training')

    # decrease learning rate
    scheduler.step()
    train_or_test(model = model,
                  data_loader = test_loader,
                  optimizer = optimizer,
                  loss_op = loss_op,
                  device = device,
                  args = args,
                  epoch = epoch,
                  mode = 'test')

    train_or_test(model = model,
                  data_loader = val_loader,
                  optimizer = optimizer,
                  loss_op = loss_op,
                  device = device,
                  args = args,
                  epoch = epoch,
                  mode = 'val')

    if epoch % args.sampling_interval == 0:
        print('......sampling......')
        sampled_images = generation_evaluation.my_sample(model, args.sample_dir, args.sample_batch_size, args.obs, sample_op)
        sample_result = {label: wandb.Image(img, caption="epoch {} - label {}".format(epoch, label)) for label, img in sampled_images.items()}

        gen_data_dir = args.sample_dir
        ref_data_dir = args.data_dir +'/test'
        paths = [gen_data_dir, ref_data_dir]
        try:
            fid_score = calculate_fid_given_paths(paths, 32, device, dims=192)
            print("Dimension {:d} works! fid score: {}".format(192, fid_score))
        except:
            print("Dimension {:d} fails!".format(192))

        if args.en_wandb:
          for label, img in sample_result.items():
            wandb.log({f"samples_{label}": img,
                        "FID": fid_score})

    if (epoch + 1) % args.save_interval == 0:
        if not os.path.exists("models"):
            os.makedirs("models")
        torch.save(model.state_dict(), 'models/{}_{}.pth'.format(model_name, epoch))


/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
100%|██████████| 259/259 [00:59<00:00,  4.38it/s]

100%|██████████| 32/32 [00:03<00:00,  8.60it/s]

100%|██████████| 32/32 [00:53<00:00,  1.68s/it]


......sampling......
Label: Class0
Label: Class1
Label: Class2
Label: Class3


Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth

  0%|          | 0.00/91.2M [00:00<?, ?B/s]
 20%|██        | 18.4M/91.2M [00:00<00:00, 192MB/s]
 52%|█████▏    | 47.2M/91.2M [00:00<00:00, 257MB/s]
100%|██████████| 91.2M/91.2M [00:00<00:00, 227MB/s]

100%|██████████| 2/2 [00:00<00:00,  4.73it/s]

100%|██████████| 17/17 [00:02<00:00,  7.12it/s]


Dimension 192 works! fid score: 28.855192436091542


100%|██████████| 259/259 [00:57<00:00,  4.51it/s]

100%|██████████| 32/32 [00:04<00:00,  6.59it/s]

100%|██████████| 259/259 [00:56<00:00,  4.59it/s]

100%|██████████| 32/32 [00:03<00:00,  8.55it/s]

100%|██████████| 259/259 [00:57<00:00,  4.53it/s]

100%|██████████| 32/32 [00:03<00:00,  8.44it/s]

100%|██████████| 259/259 [00:58<00:00,  4.39it/s]

100%|██████████| 32/32 [00:04<00:00,  7.02it/s]

100%|██████████| 259/259 [00:58<00:00,  4.44it/s]

100%|██████████| 32/32 [00:03<00:00,  8.09it/s]

100%|██████████| 32/32 [00:13<00:00,  2.33it/s]


...


100%|██████████| 259/259 [01:01<00:00,  4.18it/s]

100%|██████████| 32/32 [00:05<00:00,  6.02it/s]

100%|██████████| 259/259 [01:01<00:00,  4.20it/s]

100%|██████████| 32/32 [00:04<00:00,  7.66it/s]

100%|██████████| 259/259 [01:01<00:00,  4.21it/s]

100%|██████████| 32/32 [00:05<00:00,  6.34it/s]

100%|██████████| 259/259 [01:01<00:00,  4.23it/s]

100%|██████████| 32/32 [00:04<00:00,  7.57it/s]

100%|██████████| 259/259 [01:02<00:00,  4.13it/s]

100%|██████████| 32/32 [00:04<00:00,  7.64it/s]

100%|██████████| 32/32 [00:14<00:00,  2.28it/s]


......sampling......
Label: Class0
Label: Class1
Label: Class2
Label: Class3



100%|██████████| 2/2 [00:00<00:00,  4.65it/s]

 92%|█████████▏| 46/50 [1:45:33<11:48, 177.22s/it]

Dimension 192 works! fid score: 32.563606188866316



100%|██████████| 259/259 [01:00<00:00,  4.28it/s]

100%|██████████| 32/32 [00:04<00:00,  7.51it/s]

100%|██████████| 259/259 [01:02<00:00,  4.13it/s]

100%|██████████| 32/32 [00:05<00:00,  5.64it/s]

100%|██████████| 259/259 [01:00<00:00,  4.26it/s]

100%|██████████| 32/32 [00:04<00:00,  7.61it/s]

100%|██████████| 259/259 [01:01<00:00,  4.18it/s]

100%|██████████| 32/32 [00:05<00:00,  5.75it/s]

100%|██████████| 50/50 [1:50:57<00:00, 133.15s/it]


In [ ]:
### CONTINUE TRAINING

base_epoch = 50

model = PixelCNN(nr_resnet=5, nr_filters=80,
            input_channels=3, nr_logistic_mix=5)
model.load_state_dict(torch.load('models_backup_f40_l5_r5/pcnn_cpen455_f80_l5_r5.pth'))
model = model.to(device)

if args.load_params:
    model.load_state_dict(torch.load(args.load_params))
    print('model parameters loaded')

optimizer = optim.Adam(model.parameters(), lr=args.lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=args.lr_decay)


for epoch in tqdm(range(base_epoch, base_epoch + args.max_epochs)):
    train_or_test(model = model,
                  data_loader = train_loader,
                  optimizer = optimizer,
                  loss_op = loss_op,
                  device = device,
                  args = args,
                  epoch = epoch,
                  mode = 'training')

    # decrease learning rate
    scheduler.step()
    train_or_test(model = model,
                  data_loader = test_loader,
                  optimizer = optimizer,
                  loss_op = loss_op,
                  device = device,
                  args = args,
                  epoch = epoch,
                  mode = 'test')

    train_or_test(model = model,
                  data_loader = val_loader,
                  optimizer = optimizer,
                  loss_op = loss_op,
                  device = device,
                  args = args,
                  epoch = epoch,
                  mode = 'val')

    if epoch % args.sampling_interval == 0:
        print('......sampling......')
        sampled_images = generation_evaluation.my_sample(model, args.sample_dir, args.sample_batch_size, args.obs, sample_op)
        sample_result = {label: wandb.Image(img, caption="epoch {} - label {}".format(epoch, label)) for label, img in sampled_images.items()}

        gen_data_dir = args.sample_dir
        ref_data_dir = args.data_dir +'/test'
        paths = [gen_data_dir, ref_data_dir]
        try:
            fid_score = calculate_fid_given_paths(paths, 32, device, dims=192)
            print("Dimension {:d} works! fid score: {}".format(192, fid_score))
        except:
            print("Dimension {:d} fails!".format(192))

        if args.en_wandb:
          for label, img in sample_result.items():
            wandb.log({f"samples_{label}": img,
                        "FID": fid_score})

    if (epoch + 1) % args.save_interval == 0:
        if not os.path.exists("models"):
            os.makedirs("models")
        torch.save(model.state_dict(), 'models/{}_{}.pth'.format(model_name, epoch))


100%|██████████| 259/259 [00:55<00:00,  4.70it/s]

100%|██████████| 32/32 [00:03<00:00,  8.64it/s]

100%|██████████| 32/32 [00:13<00:00,  2.37it/s]


......sampling......
Label: Class0
Label: Class1
Label: Class2
Label: Class3



100%|██████████| 2/2 [00:00<00:00,  6.91it/s]

  2%|▏         | 1/50 [05:41<4:38:58, 341.61s/it]

Dimension 192 works! fid score: 40.82594718093446



100%|██████████| 259/259 [00:56<00:00,  4.60it/s]

100%|██████████| 32/32 [00:03<00:00,  8.88it/s]

100%|██████████| 259/259 [00:55<00:00,  4.63it/s]

100%|██████████| 32/32 [00:04<00:00,  7.35it/s]

100%|██████████| 259/259 [00:56<00:00,  4.59it/s]

100%|██████████| 32/32 [00:04<00:00,  7.73it/s]

100%|██████████| 259/259 [00:58<00:00,  4.44it/s]

100%|██████████| 32/32 [00:03<00:00,  8.50it/s]

100%|██████████| 259/259 [00:57<00:00,  4.49it/s]

100%|██████████| 32/32 [00:04<00:00,  6.99it/s]

100%|██████████| 32/32 [00:13<00:00,  2.31it/s]


...


100%|██████████| 259/259 [00:59<00:00,  4.36it/s]

100%|██████████| 32/32 [00:04<00:00,  7.55it/s]

100%|██████████| 259/259 [00:59<00:00,  4.38it/s]

100%|██████████| 32/32 [00:03<00:00,  8.02it/s]

100%|██████████| 259/259 [01:01<00:00,  4.22it/s]

100%|██████████| 32/32 [00:05<00:00,  6.00it/s]

100%|██████████| 259/259 [00:59<00:00,  4.35it/s]

100%|██████████| 32/32 [00:03<00:00,  8.19it/s]

100%|██████████| 259/259 [00:59<00:00,  4.33it/s]

100%|██████████| 32/32 [00:03<00:00,  8.03it/s]

100%|██████████| 32/32 [00:13<00:00,  2.34it/s]


......sampling......
Label: Class0
Label: Class1
Label: Class2
Label: Class3



100%|██████████| 2/2 [00:00<00:00,  6.37it/s]

 92%|█████████▏| 46/50 [1:42:56<11:26, 171.65s/it]

Dimension 192 works! fid score: 22.199479562968904



100%|██████████| 259/259 [01:00<00:00,  4.29it/s]

100%|██████████| 32/32 [00:04<00:00,  7.62it/s]

100%|██████████| 259/259 [01:01<00:00,  4.22it/s]

100%|██████████| 32/32 [00:03<00:00,  8.12it/s]

100%|██████████| 259/259 [00:59<00:00,  4.32it/s]

100%|██████████| 32/32 [00:05<00:00,  6.10it/s]

100%|██████████| 259/259 [01:00<00:00,  4.27it/s]

100%|██████████| 32/32 [00:04<00:00,  7.70it/s]

100%|██████████| 50/50 [1:48:11<00:00, 129.84s/it]


In [16]:
! python3 classification_evaluation.py -b 16 -m test

{'batch_size': 16, 'data_dir': 'data', 'mode': 'test'}
/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
100% 33/33 [00:17<00:00,  1.89it/s]
[2, 1, 2, 0, 2, 3, 2, 0, 0, 1, 0, 1, 3, 1, 0, 1, 1, 1, 1, 2, 2, 1, 3, 3, 3, 3, 0, 2, 2, 3, 1, 1, 0, 1, 0, 2, 1, 0, 3, 3, 3, 3, 1, 0, 1, 1, 3, 1, 2, 3, 2, 3, 1, 1, 0, 2, 1, 3, 0, 1, 2, 0, 0, 2, 0, 0, 3, 0, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 1, 3, 1, 2, 2, 0, 3, 0, 1, 0, 0, 1, 1, 1, 2, 0, 3, 3, 2, 0, 3, 0, 2, 3, 0, 2, 0, 0, 0, 3, 2, 2, 0, 1, 2, 0, 0, 3, 1, 1, 0, 1, 2, 3, 3, 0, 0, 1, 2, 3, 0, 2, 2, 2, 0, 2, 0, 3, 0, 0, 2, 1, 3, 2, 2, 3, 2, 2, 3, 1, 0, 2, 2, 1, 3, 0, 2, 3, 1, 3, 1, 3, 0, 3, 0, 3, 0, 0, 0, 3, 2, 3, 0, 2, 1, 2, 1, 1, 0, 3, 3, 0, 2, 2, 2, 3, 1, 1, 3, 2, 3, 3, 2, 2, 2, 0, 2, 0, 3, 1, 0, 1, 1, 0, 0, 2, 1, 3, 2, 1, 1, 3, 0, 0, 3, 3, 0, 0, 2, 0, 3, 2, 0, 1, 1, 2, 0, 1, 

In [ ]:
0

In [18]:
! python3 generation_evaluation.py

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Label: Class0
Label: Class1
Label: Class2
Label: Class3
#generated images: 100, #reference images: 519
Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:00<00:00, 126MB/s]
100% 1/1 [00:01<00:00,  1.03s/it]
100% 5/5 [00:02<00:00,  1.70it/s]
Dimension 192 works! fid score: 25.917448519048907
Average fid score: 25.917448519048907


In [ ]:
! python3 wandb_sweep.py

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: parshan-pjavanrood (parshan-pjavanrood-university-of-british-columbia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Agent Starting Run: t984sodv with config:
wandb: 	nr_filters: 80
wandb: 	nr_logistic_mix: 5
wandb: 	nr_resnet: 5
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /content/drive/MyDrive/cpen455-project/wandb/run-20250406_053238-t984sodv
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run summer-sweep-13
wandb: ⭐️ View project at https://wandb.ai/parshan-pjavanrood-university-of-british-columbia/CPEN455HW
wandb: 🧹 View sweep at https://wandb.ai/parshan-pjavanrood-university-of-british-columbia/CPEN455HW/sweeps/xf7hbyy9
wandb: 🚀 View run at https://wandb.ai/parshan-pjavanrood-university-of-british-columbia/CPEN455HW/runs/t984sodv
{'batch_size': 32,
 'data_d

In [ ]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()